In [1]:
!pip install faiss-cpu numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 23.1 MB/s eta 0:00:00


In [2]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.70.0
    Uninstalling openai-1.70.0:
      Successfully uninstalled openai-1.70.0


In [3]:
import faiss
import openai
import numpy as np

In [5]:
openai.api_key = "YOUR_OPENAI_API_KEY_HERE"

In [6]:
# Helper: Get embedding using OpenAI
def get_embedding(text):
    response = openai.Embedding.create(
        input=[text],
        model="text-embedding-ada-002"
    )
    return np.array(response['data'][0]['embedding'], dtype=np.float32)

In [7]:
# Example texts
texts = [
    "FAISS is a library for efficient similarity search.",
    "It is developed by Facebook AI Research.",
    "It supports cosine and L2 distance search.",
    "You can use FAISS with OpenAI embeddings."
]

In [8]:
# Get embeddings
embeddings = np.array([get_embedding(text) for text in texts])

In [9]:
embeddings

array([[-0.02997606,  0.02110047, -0.01385765, ...,  0.00743819,
        -0.00234973, -0.05118818],
       [-0.00988787,  0.00257791,  0.00713675, ..., -0.01343943,
        -0.01611655, -0.03309409],
       [ 0.00019641,  0.02353237, -0.00626444, ...,  0.00275607,
        -0.02853837, -0.02821943],
       [-0.01981817, -0.00651386,  0.01091175, ...,  0.00290081,
         0.01450751, -0.03783847]], dtype=float32)

In [10]:
# Normalize if using cosine similarity
faiss.normalize_L2(embeddings)  # Optional: only if you're doing cosine similarity

In [11]:
# Create FAISS index
dimension = embeddings.shape[1]
print(dimension)

1536


In [12]:
index = faiss.IndexFlatIP(dimension)

In [13]:
index

<faiss.swigfaiss_avx2.IndexFlatIP; proxy of <Swig Object of type 'faiss::IndexFlatIP *' at 0x7adb9953d9e0> >

In [14]:
# Add vectors to index
index.add(embeddings)

In [15]:
# Store original texts for lookup
text_id_map = {i: text for i, text in enumerate(texts)}

In [16]:
text_id_map

{0: 'FAISS is a library for efficient similarity search.',
 1: 'It is developed by Facebook AI Research.',
 2: 'It supports cosine and L2 distance search.',
 3: 'You can use FAISS with OpenAI embeddings.'}

In [17]:
# Query example
query = "What is FAISS?"
query_vector = get_embedding(query)
faiss.normalize_L2(query_vector.reshape(1, -1))  # Normalize for cosine
# Search top 2 most similar
k = 2
distances, indices = index.search(query_vector.reshape(1, -1), k)


In [18]:
# Display results
for i, idx in enumerate(indices[0]):
    print(f"Result {i+1}: {text_id_map[idx]} (Score: {distances[0][i]:.4f})")

Result 1: FAISS is a library for efficient similarity search. (Score: 0.8826)
Result 2: You can use FAISS with OpenAI embeddings. (Score: 0.8335)
